In [ ]:
!pip install spotipy
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling hugg

In [ ]:
# Importing libraries
import textblob
from transformers import pipeline
import gradio as gr

In [ ]:
# Text analysis
def get_sentiment(text):
    analysis = textblob.TextBlob(text)

    polarity = analysis.sentiment.polarity
    if polarity > 0:
        return 'happy'
    elif polarity < 0:
        return 'sad'
    else:
        return 'neutral'

In [ ]:
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

def get_emotion(text):
    result = emotion_classifier(text)[0]
    return result['label']0

SyntaxError: invalid syntax (<ipython-input-4-8421da76f745>, line 5)

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = 'f62ac70f21fb44d182c409a5c377db68'
client_secret = '7ac7146820d74398aa2ada86aebfde4e'

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Mapping Emotions and Genres
emotion_to_genre = {
    'joy': ['pop', 'dance', 'happy'],
    'sadness': ['acoustic', 'piano', 'sad'],
    'anger': ['rock', 'metal', 'punk'],
    'fear': ['ambient', 'electronic', 'chill'],
    'love': ['romance', 'soul', 'rnb'],
    'surprise': ['indie', 'experimental', 'alternative'],
}

In [ ]:
def get_recommendations(emotion):

    genres = emotion_to_genre.get(emotion, ['pop'])


    results = sp.recommendations(seed_genres=genres, limit=10)
    recommendations = []

    for track in results['tracks']:
        recommendations.append({
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'preview_url': track['preview_url']
        })

    return recommendations



In [ ]:
def  recommend_songs_based_on_text(text):

    emotion = get_sentiment(text)


    recommendations = get_recommendations(emotion)

    for i, song in enumerate(recommendations, 1):
        print(f"{i}. {song['name']} by {song['artist']}")
        if song['preview_url']:
             print(f"Preview: {song['preview_url']}")

In [ ]:
user_text = "tired"
recommend_songs_based_on_text(user_text)

In [ ]:
def recommend_songs_based_on_text_web(text):
    emotion = get_sentiment(text)
    recommendations = get_recommendations(emotion)
    songs_list =""
    for i in recommendations:
      if i['preview_url'] == None:
        songs_info = "Song : "+i['name'] + ' | ' + "Artist :"+i['artist'] + "Preview" + "None\n"
      else:
        songs_info = "Song : " + i['name'] + ' | '+ "Artist :"+  i['artist'] + ' | ' +"Preview"+i["preview_url"] + '\n'
      songs_list += songs_info + '\n'
      print(songs_list)
    return songs_list
print(recommend_songs_based_on_text_web("tired"))

In [ ]:
interface = gr.Interface(
    fn=recommend_songs_based_on_text_web,
    inputs=gr.Textbox(label="Describe Your Mood"),
    outputs=gr.Textbox(label="Music Recommendations"),
    title="Emotion-Based Music Recommendation System",
    description="Enter a brief description of your mood, and the system will detect your emotion and suggest songs from Spotify to match it."
)

In [ ]:
interface.launch()